# Debugging and the Python Debugger

## UBC EOAS SWC Workout
### 10-Sep-2015
### Doug Latornell <dlatornell@eos.ubc.ca>

## How to Debug

The Software Carpentry [Python Debugging lesson](http://swcarpentry.github.io/python-novice-inflammation/09-debugging.html)

* Know what it's supposed to do
  1. Write unit tests if you can
  2. Test with simplified data
  3. Test a simplified case
  4. Compare to an oracle
  5. Check conservation laws
  6. Visualize
* Make it fail every time
* Make it fail fast
* Change one thing at a time, for a reason
* Keep track of what you've done
* Be humble

## Debugging Tools

The Python Standard Library debugger [pdb](https://docs.python.org/3/library/pdb.html#module-pdb),
and it's IPython implementation [ipdb](https://pypi.python.org/pypi/ipdb)
```bash
$ pip install -U ipdb
```

* In Jupyter/IPython notebooks
* From the command-line

## 1-d Diffusion Equation

$$\frac{\partial T}{\partial t} = \kappa \frac{\partial^2 T}{\partial x^2}$$

with $T_0 = 0$ everywhere except the midpoint,
and $T = 0$ at the ends.

In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo('b4D2ktTtw7E', rel=0, width=800)

## Post-mortem Debugging

In [2]:
%matplotlib inline

In [ ]:
# %load heat_conduction.py
"""Heat conduction in a rod (i.e. 1-d diffusion over time)

Example code for 10Sep2015 EOAS SWC Workout on Debugging and pdb
"""
import matplotlib.pyplot as plt
import numpy as np


def main():
    # Diffusion coefficient
    kappa = 0.01

    # 1-d spatial grid
    x_max = 100
    x_max_p1 = x_max + 1
    mid = x_max_p1 / 2

    # Time domain
    max_time = 10
    timesteps = 50
    dt = max_time / timesteps

    # Initialize temperature
    temp = np.zeros(x_max_p1)
    dtemp = np.zeros_like(temp)
    temp[mid] = 1.

    for time in np.linspace(0, max_time, timesteps):
        # Boundary conditions
        temp[1] = 0
        temp[x_max_p1] = 0
        for i in range(x_max_p1):
            dtemp[i] = kappa * (temp[i-1] - 2*temp[i] + temp[i+1])/dt ** 2
        temp = temp + dtemp
    plot_temp_distribution(temp, time)


def plot_temp_distribution(temp, time):
    fig, ax = plt.subplots(1, 1, figsize=(16, 2))
    ax.plot(temp)
    ax.set_xlabel('x')
    ax.set_ylabel('temp')
    ax.set_title('temp distribution at time = {}'.format(time))
    return fig


if __name__ == '__main__':
    main()


Use the `%debug` magic in notebooks.

`help` shows all of the available commands,
and `help <command>` shows information for `<command>`,
or see the [Debugger Commands](https://docs.python.org/3/library/pdb.html#debugger-commands) section of the Python docs.

Use `q` or `quit` to exit the debugger before you move to another cell
because while you are in the debugger the kernel is blocked waiting for your next command.

In [ ]:
%debug

The `%pdb` magic toggles an automatic `%debug` whenever an exception is raised.

Keep debugging until the code runs without raising an exception.

Is the result correct?

## Interactive Debugging

Insert the line
```python
import ipdb; ipdb.set_trace()
```    
before the line where you want to start stepping through the code

## Debugging Modules from the Command-line

The command:
```bash
$ ipdb mymodule.py
```

for Python 2, or
```bash 
$ ipdb3 mymodule.py
```

for Python 3

prepares to run `mymodule.py` but drops into the debugger before the 1st executable line.

```bash
$ python -m ipdb mymodule.py
```

can also be used in Python 2.7 or later.

To begin execution and stop if an exception is raised use the `r` or `run` command.

Alternatively, you can
* step through the code with the `s` or `step` and `n` or `next` commands
* display values as your go or automatically with the `display` command
* set 1 or more breakpoints (perhaps with conditions) where you want to drop into the debugger again, then start execution with `r` or `run`
* etc...

See the [Debugger Commands](https://docs.python.org/3/library/pdb.html#debugger-commands) section of the Python docs.

## Using `ipdb` to Investigate Unit Test Failures

Insert
```python
import ipdb; ipdb.set_trace()
```
as the line before the call to the function under test,
for example:
```python
def test_gaps():
    '''
    check gap behaviour
    '''
    import ipdb; ipdb.set_trace()
    score = matching.matchScore('---','---')
    assert score == 0, '--- and --- should have scored 0, but got %i' % score
```
When you run the test suite execution will stop at that line 
so that you can step into the function under test with `s`  or `step`
and then explore it to figure out why the test is failing.

Don't forget to delete `import ipdb; ipdb.set_trace()` before you commit your changes;
running your test suite jsut before you commit is a good way to ensure that you have.